In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
#import foolbox
import tqdm
import imageio
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import os
from torch.utils.data import Dataset,DataLoader
import skimage.transform
import skimage
from skimage import io
import cv2
from torch.nn import functional as F

In [2]:
torch.cuda.set_device(3)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
#calcyulating the mean and the standard deviation


In [3]:
model = torchvision.models.resnet50(pretrained=True, )

In [8]:
model.eval()
model.to(DEVICE)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [9]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        
        # define the resnet152
        self.resnet = model
        
        # isolate the feature blocks
        self.features = nn.Sequential(self.resnet.conv1,
                                      self.resnet.bn1,
                                      nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
                                      self.resnet.layer1, 
                                      self.resnet.layer2, 
                                      self.resnet.layer3, 
                                      self.resnet.layer4)
        
        # average pooling layer
        self.avgpool = self.resnet.avgpool
        
        # classifier
        self.classifier1 = self.resnet.fc
       
        
        # gradient placeholder
        self.gradient = None
    
    # hook for the gradients
    def activations_hook(self, grad):
        self.gradient = grad
    
    def get_gradient(self):
        return self.gradient
    
    def get_activations(self, x):
        return self.features(x)
    
    def forward(self, x):
        
        # extract the features
        out_x = self.features(x)
        
        # register the hook
        h = out_x.register_hook(self.activations_hook)
        
        # complete the forward pass
        x = self.avgpool(out_x)
        x = x.view((1, -1))
        x = self.classifier1(x)
       
        
        return x, out_x

In [10]:
mean = np.array([0.5, 0.5, 0.5]).reshape((3, 1, 1))
std = np.array([0.5, 0.5, 0.5]).reshape((3, 1, 1))
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)


In [15]:
class cam_dataset(Dataset):
    def __init__(self,img_transform1):
        #/workspace/data_fine_grain/cub200/raw/CUB_200_2011/(base) ironman@bigbox:~$ CUB_200_2011
        self.dir_path = './imagenet/fgsm/'
        self.files_list = []
        self.original_label = []
        self.misclassified_label = []
        self.img_transform1 = img_transform1
        
        for r, d, f in os.walk(self.dir_path):
            for file in f:
                if '.png' in file and file != "fgsm_16-checkpoint.png":
                    #print("here")
                    #print(file)
                    #print(file.split('.')[-2].split('_')[-2])
                    #print(file.split('.')[-2].split('_')[-1])
                    
                    self.files_list.append(os.path.join(r,file))
                    self.original_label.append(int(file.split('.')[-2].split('_')[-2]))
                    self.misclassified_label.append(int(file.split('.')[-2].split('_')[-1]))

        
            
    def __getitem__(self,index):
        #if(self.train):
           
            img1_path = self.files_list[index]
            img1_path = img1_path.rstrip()
            img1 = io.imread(img1_path)
            
            #print(img1.shape)
            if(img1.shape.__len__() == 2):
                img1 = np.repeat(img1[:, :, np.newaxis], 3, axis=2) 
            #print(img1_path)
            #plt.imshow(img1)
            img1 = img1.astype(np.float)
            img1 = cv2.resize(img1,(128,128),interpolation = cv2.INTER_AREA)
           
            img1 = self.img_transform1(img1)
            label = self.original_label[index]
            label = int(label)
            mislabel = self.misclassified_label[index]
            mislabel = int(mislabel)
            return {"img": img1, "label": label,"mislabel":mislabel,"path": img1_path}
   
        #return img1,img2,label
    def __len__(self):
        return len(self.original_label)


test_set = cam_dataset(transform)

testloader = DataLoader(test_set,batch_size =1,shuffle = True, num_workers =0)

In [16]:
data = test_set[0]

In [22]:
#plt.imshow(data["img"])
for data in testloader:
    #net1 = net1.to(DEVICE)
    #net2 = net2.to(DEVICE)
    img1 = data['img'].to(DEVICE)
    print("check the image")
    print(img1.shape)
    path = data['path']

    #img1 = img1.float()
    img1 = img1.view(-1,img1.shape[-3],img1.shape[-2],img1.shape[-1])
    print(img1.shape)
    img1 = img1.float()
    #print(img1.shape)
    label = data['label'].to(DEVICE)
    mislabel = data['mislabel'].to(DEVICE)
   
    resnet = ResNet()
    resnet = resnet.to(DEVICE)
    resnet.eval()
    print("check the image sizes")
    print(img1.shape)
    img1 = Variable(img1.cuda(), requires_grad=True)
    pred, acts = resnet(img1)
    max_index = pred.argmax(dim=1) 
    print("check the max index")
    print(max_index)
    ##############################################################for max_index
    pred[:,(max_index.item()-1)].backward()
    gradients = resnet.get_gradient()
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    activations = resnet.get_activations(img1).detach()
    for i in range(activations.shape[1]):
        activations[:, i, :, :] *= pooled_gradients[i]
    heatmap = F.relu(torch.sum(activations, dim=1)).squeeze()
    heatmap = np.maximum(heatmap.cpu(), 0)
    heatmap /= torch.max(heatmap)
    #plt.matshow(heatmap.squeeze())
    heatmap = heatmap.numpy()
    #print(np.unique(heatmap))
    p = np.percentile(heatmap, 97)
    heatmap[heatmap>=p]=1
    heatmap[heatmap<p]=0
    print(np.unique(heatmap))
    print("CHECK THE PATH")
    print(path[0])
    print(label.item())
    img = io.imread(path[0])
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * 0.4 + img
    name = path[0].split('/')[-1].split('.')[-2]
    print("check the name")
    print(name)
    
    cv2.imwrite('./maxlabel_camfiles/'+str(name)+'_'+str(max_index.item()+1)+'.jpg', superimposed_img)
    #########################################################################for label
   
    
    
    

check the image
torch.Size([1, 3, 128, 128])
torch.Size([1, 3, 128, 128])
check the image sizes
torch.Size([1, 3, 128, 128])
check the max index
tensor([818], device='cuda:2')
[0. 1.]
CHECK THE PATH
./imagenet/fgsm/fgsm_10_0_389.png
0
check the name
fgsm_10_0_389
check the image
torch.Size([1, 3, 128, 128])
torch.Size([1, 3, 128, 128])
check the image sizes
torch.Size([1, 3, 128, 128])
check the max index
tensor([818], device='cuda:2')
[0. 1.]
CHECK THE PATH
./imagenet/fgsm/fgsm_10817_431_452.png
431
check the name
fgsm_10817_431_452
check the image
torch.Size([1, 3, 128, 128])
torch.Size([1, 3, 128, 128])
check the image sizes
torch.Size([1, 3, 128, 128])
check the max index
tensor([818], device='cuda:2')
[0. 1.]
CHECK THE PATH
./imagenet/fgsm/fgsm_17012_673_882.png
673
check the name
fgsm_17012_673_882
check the image
torch.Size([1, 3, 128, 128])
torch.Size([1, 3, 128, 128])
check the image sizes
torch.Size([1, 3, 128, 128])
check the max index
tensor([818], device='cuda:2')
[0. 1.]


for data in testloader:
    #net1 = net1.to(DEVICE)
    #net2 = net2.to(DEVICE)
    img1 = data['img'].to(DEVICE)
    print("check the image")
    print(img1.shape)
    path = data['path']

    #img1 = img1.float()
    img1 = img1.view(-1,img1.shape[-3],img1.shape[-2],img1.shape[-1])
    print(img1.shape)
    img1 = img1.float()
    #print(img1.shape)
    label = data['label'].to(DEVICE)
    print("CHECK THE PATH")
    print(path[0])   
    resnet = ResNet()
    resnet = resnet.to(DEVICE)
    resnet.eval()
    print("check the image sizes")
    print(img1.shape)
    img1 = Variable(img1.cuda(), requires_grad=True)
    pred, acts = resnet(img1)
    max_index = pred.argmax(dim=1) 
    print("check the max index")
    print(max_index)
    pred[:,(max_index.item())].backward()
    gradients = resnet.get_gradient()
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    activations = resnet.get_activations(img1).detach()
    for i in range(activations.shape[1]):
        activations[:, i, :, :] *= pooled_gradients[i]
    heatmap = F.relu(torch.sum(activations, dim=1)).squeeze()
    heatmap = np.maximum(heatmap.cpu(), 0)
    heatmap /= torch.max(heatmap)
    #plt.matshow(heatmap.squeeze())
    heatmap = heatmap.numpy()
    #print(np.unique(heatmap))
    
    print(np.unique(heatmap))
    
    print(label.item())
    img = io.imread(path[0])
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    p = np.percentile(heatmap, 70)
    heatmap[heatmap>=p]=1
    heatmap[heatmap<p]=0
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * 0.4 + img
    name = path[0].split('/')[-1].split('.')[-2]
    cv2.imwrite('./originallabel_camfiles/'+str(name)+'_'+str(max_index.item()+1)+'.jpg', superimposed_img)
    

In [24]:
#plt.imshow(data["img"])
for data in testloader:
    #net1 = net1.to(DEVICE)
    #net2 = net2.to(DEVICE)
    img1 = data['img'].to(DEVICE)
    print("check the image")
    print(img1.shape)
    path = data['path']

    #img1 = img1.float()
    img1 = img1.view(-1,img1.shape[-3],img1.shape[-2],img1.shape[-1])
    print(img1.shape)
    img1 = img1.float()
    #print(img1.shape)
    label = data['label'].to(DEVICE)
    mislabel = data['mislabel'].to(DEVICE)
   
    resnet = ResNet()
    resnet = resnet.to(DEVICE)
    resnet.eval()
    print("check the image sizes")
    print(img1.shape)
    img1 = Variable(img1.cuda(), requires_grad=True)
    pred, acts = resnet(img1)
    max_index = pred.argmax(dim=1) 
    print("check the max index")
    print(max_index)
    ##############################################################for actual label
    pred[:,(label.item())].backward()
    gradients = resnet.get_gradient()
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    activations = resnet.get_activations(img1).detach()
    for i in range(activations.shape[1]):
        activations[:, i, :, :] *= pooled_gradients[i]
    heatmap = F.relu(torch.sum(activations, dim=1)).squeeze()
    heatmap = np.maximum(heatmap.cpu(), 0)
    heatmap /= torch.max(heatmap)
    #plt.matshow(heatmap.squeeze())
    heatmap = heatmap.numpy()
    #print(np.unique(heatmap))
    p = np.percentile(heatmap, 97)
    heatmap[heatmap>=p]=1
    heatmap[heatmap<p]=0
    print(np.unique(heatmap))
    print("CHECK THE PATH")
    print(path[0])
    print(label.item())
    img = io.imread(path[0])
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * 0.4 + img
    name = path[0].split('/')[-1].split('.')[-2]
    print("check the name")
    print(name)
    
    cv2.imwrite('./originallabel_camfiles/'+str(name)+'_'+str(label.item())+'.jpg', superimposed_img)
    #########################################################################for label
   

check the image
torch.Size([1, 3, 128, 128])
torch.Size([1, 3, 128, 128])
check the image sizes
torch.Size([1, 3, 128, 128])
check the max index
tensor([818], device='cuda:2')
[0. 1.]
CHECK THE PATH
./imagenet/fgsm/fgsm_33_1_46.png
1
check the name
fgsm_33_1_46
check the image
torch.Size([1, 3, 128, 128])
torch.Size([1, 3, 128, 128])
check the image sizes
torch.Size([1, 3, 128, 128])
check the max index
tensor([818], device='cuda:2')
[0. 1.]
CHECK THE PATH
./imagenet/fgsm/fgsm_11699_465_419.png
465
check the name
fgsm_11699_465_419
check the image
torch.Size([1, 3, 128, 128])
torch.Size([1, 3, 128, 128])
check the image sizes
torch.Size([1, 3, 128, 128])
check the max index
tensor([818], device='cuda:2')
[0. 1.]
CHECK THE PATH
./imagenet/fgsm/fgsm_30_1_29.png
1
check the name
fgsm_30_1_29
check the image
torch.Size([1, 3, 128, 128])
torch.Size([1, 3, 128, 128])
check the image sizes
torch.Size([1, 3, 128, 128])
check the max index
tensor([818], device='cuda:2')
[0. 1.]
CHECK THE PATH
.

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in greater_equal
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in less


[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
CHECK THE PATH
./imagenet/fgsm/fgsm_3_0_793.png
0
check the name
fgsm_3_0_793
check the image
torch.Size([1, 3, 128, 128])
torch.Size([1, 3, 128, 128])
check the image sizes
torch.Size([1, 3, 128, 128])
check the max index
tensor([818], device='cuda:2')
[0. 1.]
CHECK THE PATH
./imagenet/fgsm/fgsm_17012_673_882.png
673
check the name
fgsm_17012_673_882
check the image
torch.Size([1, 3, 128, 128])
torch.Size([1, 3, 128, 128])
check the image sizes
torch.Size([1, 3, 128, 128])
check the max index
tensor([818], device='cuda:2')
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
CHECK THE PATH
./imagenet/fgsm/fgsm_4262_172_203.png
172
check the name
fgsm_4262_172_203
check the image
torch.Size([1, 3, 128, 128])
torch.Size([1, 3, 128, 128])
check the image sizes
torch.Size([1, 3, 128, 128])
check the max index
tensor([818], device='cuda:2')
[0. 1.]
CHECK THE PATH
./imagenet/fgsm/fgsm_44_1_29.png
1
check the name

In [ ]:
|